In [1]:
from tensorflow.compat.v1 import ConfigProto
from tensorflow.compat.v1 import InteractiveSession

config = ConfigProto()
config.gpu_options.per_process_gpu_memory_fraction = 0.2
config.gpu_options.allow_growth = True
session = InteractiveSession(config=config)

In [2]:
from tensorflow.keras.layers import Input, Lambda, Dense, Flatten
from tensorflow.keras.models import Model
#from tensorflow.keras.applications.resnet152V2 import ResNet152V2
#from keras.applications.vgg16 import VGG16
from tensorflow.keras.applications.inception_v3 import preprocess_input
from tensorflow.keras.preprocessing import image
from tensorflow.keras.preprocessing.image import ImageDataGenerator,load_img
from tensorflow.keras.models import Sequential
import numpy as np
from glob import glob

In [3]:
IMAGE_SIZE = [256, 256]

train_path = 'C:\\Users\\Priyanka\\Desktop\\6th_sem_project\\final-dataset\\training'
valid_path = 'C:\\Users\\Priyanka\\Desktop\\6th_sem_project\\final-dataset\\test'

In [4]:
import tensorflow
resnet152V2 =tensorflow.keras.applications.ResNet152V2(input_shape=IMAGE_SIZE + [3], weights='imagenet', include_top=False)

234553344/234545216 [==============================] - 47s 0us/step


In [5]:
for layer in resnet152V2.layers:
    layer.trainable = False

In [6]:
folders = glob('C:\\Users\\Priyanka\\Desktop\\6th_sem_project\\final-dataset\\training\\*')

In [7]:
x = Flatten()(resnet152V2.output)

In [8]:
prediction = Dense(len(folders), activation='softmax')(x)

# create a model object
model = Model(inputs=resnet152V2.input, outputs=prediction)

In [9]:
model.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 256, 256, 3) 0                                            
__________________________________________________________________________________________________
conv1_pad (ZeroPadding2D)       (None, 262, 262, 3)  0           input_1[0][0]                    
__________________________________________________________________________________________________
conv1_conv (Conv2D)             (None, 128, 128, 64) 9472        conv1_pad[0][0]                  
__________________________________________________________________________________________________
pool1_pad (ZeroPadding2D)       (None, 130, 130, 64) 0           conv1_conv[0][0]                 
______________________________________________________________________________________________

__________________________________________________________________________________________________
conv4_block4_out (Add)          (None, 16, 16, 1024) 0           conv4_block3_out[0][0]           
                                                                 conv4_block4_3_conv[0][0]        
__________________________________________________________________________________________________
conv4_block5_preact_bn (BatchNo (None, 16, 16, 1024) 4096        conv4_block4_out[0][0]           
__________________________________________________________________________________________________
conv4_block5_preact_relu (Activ (None, 16, 16, 1024) 0           conv4_block5_preact_bn[0][0]     
__________________________________________________________________________________________________
conv4_block5_1_conv (Conv2D)    (None, 16, 16, 256)  262144      conv4_block5_preact_relu[0][0]   
__________________________________________________________________________________________________
conv4_bloc

conv4_block10_2_pad (ZeroPaddin (None, 18, 18, 256)  0           conv4_block10_1_relu[0][0]       
__________________________________________________________________________________________________
conv4_block10_2_conv (Conv2D)   (None, 16, 16, 256)  589824      conv4_block10_2_pad[0][0]        
__________________________________________________________________________________________________
conv4_block10_2_bn (BatchNormal (None, 16, 16, 256)  1024        conv4_block10_2_conv[0][0]       
__________________________________________________________________________________________________
conv4_block10_2_relu (Activatio (None, 16, 16, 256)  0           conv4_block10_2_bn[0][0]         
__________________________________________________________________________________________________
conv4_block10_3_conv (Conv2D)   (None, 16, 16, 1024) 263168      conv4_block10_2_relu[0][0]       
__________________________________________________________________________________________________
conv4_bloc

conv4_block33_3_conv (Conv2D)   (None, 16, 16, 1024) 263168      conv4_block33_2_relu[0][0]       
__________________________________________________________________________________________________
conv4_block33_out (Add)         (None, 16, 16, 1024) 0           conv4_block32_out[0][0]          
                                                                 conv4_block33_3_conv[0][0]       
__________________________________________________________________________________________________
conv4_block34_preact_bn (BatchN (None, 16, 16, 1024) 4096        conv4_block33_out[0][0]          
__________________________________________________________________________________________________
conv4_block34_preact_relu (Acti (None, 16, 16, 1024) 0           conv4_block34_preact_bn[0][0]    
__________________________________________________________________________________________________
conv4_block34_1_conv (Conv2D)   (None, 16, 16, 256)  262144      conv4_block34_preact_relu[0][0]  
__________

In [10]:
model.compile(
  loss='categorical_crossentropy',
  optimizer='adam',
  metrics=['accuracy']
)

In [11]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

train_datagen = ImageDataGenerator(rescale = 1./255,
                                   shear_range = 0.2,
                                   zoom_range = 0.2,
                                   horizontal_flip = True)

test_datagen = ImageDataGenerator(rescale = 1./255)

In [13]:
training_set = train_datagen.flow_from_directory('C:\\Users\\Priyanka\\Desktop\\6th_sem_project\\final-dataset\\training',
                                                 target_size = (256, 256),
                                                 batch_size = 32,
                                                 class_mode = 'categorical')

Found 12687 images belonging to 7 classes.


In [14]:
test_set = test_datagen.flow_from_directory('C:\\Users\\Priyanka\\Desktop\\6th_sem_project\\final-dataset\\test',
                                            target_size = (256, 256),
                                            batch_size = 32,
                                            class_mode = 'categorical')

Found 331 images belonging to 7 classes.


In [15]:
r = model.fit_generator(
  training_set,
  validation_data=test_set,
  epochs=20,
  steps_per_epoch=len(training_set),
  validation_steps=len(test_set)
)

Instructions for updating:
Please use Model.fit, which supports generators.
Epoch 1/20
397/397 [==============================] - 7124s 18s/step - loss: 1.4688 - accuracy: 0.9184 - val_loss: 1.4382 - val_accuracy: 0.9396
Epoch 2/20
397/397 [==============================] - 8835s 22s/step - loss: 0.9306 - accuracy: 0.9596 - val_loss: 0.6159 - val_accuracy: 0.9819
Epoch 3/20
397/397 [==============================] - 9184s 23s/step - loss: 0.8074 - accuracy: 0.9706 - val_loss: 0.7678 - val_accuracy: 0.9668
Epoch 4/20
397/397 [==============================] - 9180s 23s/step - loss: 0.6431 - accuracy: 0.9779 - val_loss: 0.3333 - val_accuracy: 0.9849
Epoch 5/20
397/397 [==============================] - 8051s 20s/step - loss: 0.6195 - accuracy: 0.9769 - val_loss: 1.3500 - val_accuracy: 0.9577
Epoch 6/20
397/397 [==============================] - 6424s 16s/step - loss: 0.5589 - accuracy: 0.9812 - val_loss: 0.9700 - val_accuracy: 0.9819
Epoch 7/20
397/397 [==============================] - 